## RAG Application With GPT-4o

In [1]:
from dotenv import load_dotenv

load_dotenv()
import os
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

#### 
1. Get a Data Loader

In [2]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
data = loader.load()
data

[Document(metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.', 'language': 'en'}, page_content="\n\n\n\n\nLangSmith User Guide | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingThis is outdated documentation for 🦜️🛠️ LangSmith, which is no longer actively maintained.For up-to-date doc

#### 
2. Convert data to Vector Database

In [4]:
from langchain_objectbox.vectorstores import ObjectBox ##vector Database
from langchain_openai import OpenAIEmbeddings

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(data)

In [6]:
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.', 'language': 'en'}, page_content='LangSmith User Guide | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingThis is outdated documentation for 🦜️🛠️ LangSmith, which is no longer actively maintained.For up-to-date documentation

In [7]:
from langchain_openai import OpenAIEmbeddings
vector = ObjectBox.from_documents(documents, OpenAIEmbeddings(), embedding_dimensions=768)
vector

#### 
3. Make a RAG pipeline

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain import hub

In [9]:
llm = ChatOpenAI(model="gpt-4o") ## Calling Gpt-4o
prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [10]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [11]:

qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vector.as_retriever(),
        chain_type_kwargs={"prompt": prompt}
    )

In [12]:
question = "Explain what is langsmith"
result = qa_chain({"query": question })
result

C:\Users\Sahana Kommalapati\AppData\Local\Temp\ipykernel_4212\858897572.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question })


{'query': 'Explain what is langsmith',
 'result': 'LangSmith is a platform designed for the development, monitoring, and testing of LLM (Large Language Model) applications. It supports various stages of the application lifecycle, offering tools for prototyping, debugging, testing, and production monitoring. Features include tracing, evaluation, automation, and playground environments for rapid experimentation and iteration.'}

In [13]:

result["result"]

'LangSmith is a platform designed for the development, monitoring, and testing of LLM (Large Language Model) applications. It supports various stages of the application lifecycle, offering tools for prototyping, debugging, testing, and production monitoring. Features include tracing, evaluation, automation, and playground environments for rapid experimentation and iteration.'

In [14]:
import pprint
pp = pprint.PrettyPrinter(indent=5)
pp.pprint(result["result"])

('LangSmith is a platform designed for the development, monitoring, and '
 'testing of LLM (Large Language Model) applications. It supports various '
 'stages of the application lifecycle, offering tools for prototyping, '
 'debugging, testing, and production monitoring. Features include tracing, '
 'evaluation, automation, and playground environments for rapid '
 'experimentation and iteration.')


In [15]:
question = "Explain Monitoring and A/B Testing in langsmith"
result = qa_chain({"query": question })
result

{'query': 'Explain Monitoring and A/B Testing in langsmith',
 'result': 'In LangSmith, monitoring involves using charts to track key metrics over time and drilling down into specific data points for debugging, especially in production. This includes tagging and grouping metadata to differentiate version performances, aiding in A/B testing by allowing users to compare changes in prompt, model, or retrieval strategies side-by-side. Together, these tools help maintain and improve LLM application performance throughout different phases of development.'}